In [1]:
from typing import List, Dict, Tuple, Set
import numpy as np
import time
from dataclasses import dataclass
import time
from datetime import date
from collections import deque
import pandas as pd
from pathlib import Path
from enum import Enum
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
from io import StringIO

# CHROME_WEBDRIVER_PATH = "/home/fabrice/Documents/PROGRAMMING/ChromeWebDrive/chromedriver"
CHROME_WEBDRIVER_PATH = "/usr/lib/chromium-browser/chromedriver"

# For retries
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Local files.
import constants as c
from website_crawler import WebCrawler

In [2]:
%pip install selenium


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install html5lib


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Data Grabber

In [4]:
df_competition_links = pd.read_csv("results/competition_links.csv")
df_competition_links

,Tournament link,Base url,Crawl date,Tournament id,Processed
0,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,9,0
1,https://ergebnisse.dancecomp.de/2022/28-0207_w...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,10,0
2,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,11,0
3,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,12,0
4,https://www.gruen-weiss-aachen.de/files/ergebn...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,13,0
...,...,...,...,...,...
218,https://www.hessen-tanzt.de/media/ht2023/56-13...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,227,0
219,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,228,0
220,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,229,0
221,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,230,0


In [5]:
print("URLs of already processed competitions:")
df_competition_links.loc[df_competition_links['Processed'] == 1]

URLs of already processed competitions:


,Tournament link,Base url,Crawl date,Tournament id,Processed


In [6]:
n = len(df_competition_links)
n_processed = len(df_competition_links.loc[df_competition_links['Processed'] == 1])
print(f"Nr. of crawled   competitions:  {n}\n"\
      f"Nr. of processed competitions:  {n_processed}\t({n_processed/n*100:0.2f} %)")

Nr. of crawled   competitions:  223
Nr. of processed competitions:  0	(0.00 %)


In [7]:
# Select the links and the ids of unprocessed tournaments.
unprocessed_tournaments_df = df_competition_links[df_competition_links['Processed'] == 0][[c.C.TOUR_LINK.value, c.C.ID.value]]

for competition_url, competition_id in unprocessed_tournaments_df.itertuples(index=False):
    print(f"Id: {competition_id}\tURL: {competition_url}")

Id: 9	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/3le-sen1sstd/index.htm
Id: 10	URL: https://ergebnisse.dancecomp.de/2022/28-0207_wdsfopenstdsen3/index.htm
Id: 11	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2022/dm_hgrsstd/index.htm
Id: 12	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/glinde-sen3sstd/index.htm
Id: 13	URL: https://www.gruen-weiss-aachen.de/files/ergebnisse/2023/2023-06-3le/3-1706_rl_hgrslat/index.htm
Id: 14	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/g55-2/index.htm
Id: 15	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/l66-4/index.htm
Id: 16	URL: https://berlin-dance-festival.de/files/bdf/results/2023/5-0104_wdsfopenstdjun2/index.htm
Id: 17	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2020/02-22.Deutsche.Meisterschaft.Junioren.II.B.Latein/index.htm
Id: 18	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/dpsen3slat/index.htm
Id: 19	URL: https://www.tanzsport.de/files/

In [8]:
url_id = 0
competition_url: str = unprocessed_tournaments_df[c.C.TOUR_LINK.value][url_id]
competition_id: int  = unprocessed_tournaments_df[c.C.ID.value][url_id]
# Start a Chrome driver to open the selected url.
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(competition_url)

# From the tournament link - extract the competition links.
# menu_url = re.compile('src="(.*)"').search(html).group(1)
competition_menue_links = [competition_url.replace('index.htm', "menu.htm#id2")]
df_tables = []
for l in competition_menue_links:
    driver.get(l)
    # Wait a second since the menu has to build itself before it can be collected.
    time.sleep(1)
        
    html = driver.page_source
    df_tables.append(pd.read_html(StringIO(html)))

# It may be that the data is stored in a nested DataFrame. 
# In this case extract it.
if len(df_tables) == 1:
    df_tables = df_tables[0]

# Since the download is complete now, close the driver.
driver.close()

print(competition_url)
df_tables

https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/3le-sen1sstd/index.htm


[                                            0  1
 0  DTV-Ranglistenturnier Sen I S Std - Aachen  ≡,
                    0                                                  1
 0      Veranstalter:               TSC Grün-Weiß Aquisgrana Aachen e.V.
 1        Ausrichter:               TSC Grün-Weiß Aquisgrana Aachen e.V.
 2     Turnierleiter:     Guido Gehlhaar TSC Grün-Weiß Aquisgrana Aachen
 3         Beisitzer:     Ilona Gehlhaar TSC Grün-Weiß Aquisgrana Aachen
 4         Protokoll:        Tobias Hock TSC Grün-Weiß Aquisgrana Aachen
 5       Turnierbüro:  Ursula Belten TSC Grün-Weiß Aquisgrana Aachen ...
 6             Musik:  Kerstin Fasel TSC Grün-Weiß Aquisgrana Aachen ...
 7   Wertungsrichter:                                                NaN
 8                 A:  Petra Matschullat-Horn TD Tanzsportclub Düssel...
 9                 B:                 Christoph Losensky Tanzclub Bernau
 10                C:  Madeleine Rohne TSA im Moritzburger Sportverei...
 11                D:  

In [9]:
# Create a folder to store the tables. Save a table and an equivalent README.md, containing the website link and the crawl date.

competition_dir = Path('results', 'comp_tables', f"{competition_id}")
# If directories do not exist, create them.
competition_dir.mkdir(parents=True, exist_ok=True)

# Save key information into a DataFrame and save it in the dir.
crawl_date = date.today().strftime("%B %d, %Y")
df_readme = pd.DataFrame({'Competition url': [competition_url],
                          'Crawl date': [crawl_date]})
df_readme.to_csv(Path(competition_dir, 'README.csv'))
# Save key information into a Markdown file and save it in the dir.
with open(Path(competition_dir, 'README.md'), 'w') as f:
    f.write(f'[Competition URL]({competition_url})\n')
    f.write(f'Crawled on {crawl_date}\n')
    f.write('by Fabrice Beaumont')

In [10]:
# Parse all tables.
# List of possible data tables:
class data_dfs(Enum):
    general_info: str   = 'General informations'
    adjudicators: str   = 'Wertungsrichter'
    couples: str        = 'Coupes'
    ranking_list: str   = 'Ergebnisliste'
    finals: str         = 'Eindrundentabelle'
    qualifications: str = 'Wertungstabelle gesamt'

df_general_info: pd.DataFrame   = None
df_couples: pd.DataFrame        = None
df_finals: pd.DataFrame         = None
df_dict: Dict[str, List[pd.DataFrame]] = {
    data_dfs.general_info: [],
    data_dfs.adjudicators: None,
    data_dfs.couples: [],
    data_dfs.ranking_list: [],
    data_dfs.finals: [],
    data_dfs.qualifications: [],
}

# The first table is expected to be the competiton name in all cases.
competition_name = df_tables[0][0][0]

for i in range(1, len(df_tables)):
    table = df_tables[i]

    # Test for different criteria to sort the tables into the dictionary.
    # GENERAL INFORMATION table.
    if table[0][0] in ['Veranstalter:', 'Ausrichter:', 'Organizer:', 'Master of Ceremony:']:
        print(f"Found general info\t\tin table {i}")
        df_dict[data_dfs.general_info].append(table)
        continue
    
    # RANKING LIST table - finals section.
    if len(table[0]) > 1 and str(table[0][0]) in ['Endrunde', 'Final']:
        print(f"Found ranking list (finals)\tin table {i}")
        df_dict[data_dfs.ranking_list].append(table)
        continue

    # RANKING LIST table - all qualification rounds sections.
    if len(table[0]) > 1 and str(table[0][1])[-5:] in ["runde"]:
        print(f"Found ranking list (round)\tin table {i}")
        df_dict[data_dfs.ranking_list].append(table)
        continue
    
    # FINALS table (containing the scating system and placements for each couple from each adjudicator).
    if len(table[0]) > 1 and str(table[1][0]) in ['Wertungsrichter']:
        print(f"Found finals scating\t\tin table {i}")
        df_dict[data_dfs.finals].append(table)
        continue
    
    # QUALIFICATIONS table (containing points for all couples).
    if len(table[0]) > 1 and str(table[0][1]) in ['Wertungsrichter']:
        print(f"Found qualifications\t\tin table {i}")
        df_dict[data_dfs.qualifications].append(table)
        continue

    print(table)

Found general info		in table 1
Found ranking list (finals)	in table 2
Found ranking list (round)	in table 3
Found finals scating		in table 4
Found qualifications		in table 5
Found qualifications		in table 6


## Processing of the general information and adjudicator list

In [11]:
df_general_info = df_dict[data_dfs.general_info][0].copy()
# Split the gereral information table into the tournament information and the adjudicator list.
split_id = np.where(df_general_info.iloc[:, 1].isnull().values == True)[0][0]
# Extract the remaining rows as information about the adjudicators.
adjudicator_names = list(df_general_info.iloc[split_id+1:, 1])
adjudicator_ids = [x[:1] for x in list(df_general_info.iloc[split_id+1:, 0])]
adjudicators = dict(zip(adjudicator_ids, adjudicator_names))

# Extract the data into individual variables.
df_general_info = pd.DataFrame(df_general_info.iloc[:split_id, :])
df_general_info.set_index([0])
df_general_info.columns = [c.C.CATEGORY.value, c.C.VALUE.value]
df_general_info = df_general_info[1:]
df_dict[data_dfs.general_info] = df_general_info
df_dict[data_dfs.general_info].to_csv(Path(competition_dir, 'general_info.csv'))
df_dict[data_dfs.general_info]


,Category,Value
1,Ausrichter:,TSC Grün-Weiß Aquisgrana Aachen e.V.
2,Turnierleiter:,Guido Gehlhaar TSC Grün-Weiß Aquisgrana Aachen
3,Beisitzer:,Ilona Gehlhaar TSC Grün-Weiß Aquisgrana Aachen
4,Protokoll:,Tobias Hock TSC Grün-Weiß Aquisgrana Aachen
5,Turnierbüro:,Ursula Belten TSC Grün-Weiß Aquisgrana Aachen ...
6,Musik:,Kerstin Fasel TSC Grün-Weiß Aquisgrana Aachen ...


In [12]:
# Process the dict of adjudicators.
adjudicators_id_club: Dict[str, Tuple[Tuple[str, str], str]] = dict()
for adjudicator_id, adjudicator_and_club in adjudicators.items():
    ac = adjudicator_and_club.split(" ")
    adjudicator, club = tuple(ac[0:2]), " ".join(ac[2:])
    
    adjudicators_id_club[adjudicator_id] = tuple([adjudicator, club])

# Create a DataFrame to hold the adjudicator information.
df_adjudicators = pd.DataFrame(columns=[c.C.SURNAME.value, c.C.NAME.value, c.C.CLUB.value])
# Add adjudicators which are unknown (defined as surname, name, club triple).
for _, adjudicator_data in adjudicators_id_club.items():
    name, surname = adjudicator_data[0]
    club = adjudicator_data[1]
    
    # Check if adjudicator is already known:    
    x = df_adjudicators.loc[(df_adjudicators[c.C.SURNAME.value] == surname) & (df_adjudicators[c.C.NAME.value] == name) & (df_adjudicators[c.C.CLUB.value] == club)]
    # Only add, if not already known.
    if len(x) == 0:   
        new_row = pd.DataFrame({c.C.SURNAME.value:surname, c.C.NAME.value:name, c.C.CLUB.value:club}, index=[0])
        df_adjudicators = pd.concat([new_row,df_adjudicators.loc[:]]).reset_index(drop=True)

df_dict[data_dfs.adjudicators] = df_adjudicators
df_dict[data_dfs.adjudicators].to_csv(Path(competition_dir, 'adjudicators.csv'))
df_dict[data_dfs.adjudicators]

,Surname,Name,Club
0,Lachenmaier,Bernd,TSA d. TSV Unterhaching 1910
1,Neuhaus,Susanne,TSG Bremerhaven
2,Voß,Peter,TSC Casino Oberalster Hamburg
3,Dres,Petra,TSC Rot-Gold Sinsheim
4,Rohne,Madeleine,TSA im Moritzburger Sportverein 1990
5,Losensky,Christoph,Tanzclub Bernau
6,Matschullat-Horn,Petra,TD Tanzsportclub Düsseldorf Rot-Weiß


In [13]:
nr_adjudicators = len(df_dict[data_dfs.adjudicators])
nr_adjudicators

7

## Processing of the coupes list

In [14]:
if len(df_dict[data_dfs.couples]) > 0:
    df_couples = df_dict[data_dfs.couples]
    # Use the first row as column names and then delete it.
    df_couples.columns = df_couples.iloc[0]
    df_couples = df_couples[1:]
    df_couples = df_couples.set_index(c.C.NR.value)
    # Extract the names of the lady and the man individually.
    # Do not separate the fist and last names since depending on multiple name and titles (e.g. 'Dr.', 'Prof.') this may be more complicated.
    # To identify a person simply test if the surname is part of the name string.
    df_couples[[c.C.MAN.value, c.C.LADY.value]] = df_couples[c.C.COUPLE.value].str.split(" / ", expand = True)
    # Rearrange the columns.
    df_couples = df_couples[[c.C.MAN.value, c.C.LADY.value, c.C.COUPLE.value, c.C.CLUB.value]]
    df_dict[data_dfs.couples] = df_couples

df_dict[data_dfs.couples]

[]

## Processing of the ranking list

In [15]:
df_dict[data_dfs.ranking_list][0]

,0,1,2,3,4,5,6,7
0,Endrunde,Endrunde,Endrunde,Endrunde,Endrunde,Endrunde,Endrunde,Endrunde
1,Platz,Paar/Club,LW,TG,WW,SF,QU,PZ
2,1.,"Spörl, Sebastian / Rodionova, Kristina (6) TSC...",1212111 1.0,1112111 1.0,1212121 1.0,1211111 1.0,1112111 1.0,5.0
3,2.,"Brinkmann, Christian / Ostholt, Alena (16) Die...",2121553 2.0,3221553 2.0,2121654 2.0,3122654 2.0,5221453 2.0,10.0
4,3.,"Otto, Sascha / Vollmer, Janne (7) TC Der Frank...",4443222 3.0,4545242 3.0,5454232 4.0,5443242 4.0,6543242 3.0,17.0
5,4.,"Ciomber, Dennis / von Brühl, Julia (2) Art of ...",5654345 4.0,6663336 6.0,4633343 3.0,4634333 3.0,4654335 5.0,21.0
6,5.,"Bohn, Bruno / Bohn, Monika (12) TC Schwarz-Wei...",6365634 5.5,5354625 5.0,6365415 5.0,6355425 5.0,3366524 4.0,24.5
7,6.,"Rybka, Harald / Rybka-Schmidt, Sarah (14) TC R...",3536466 5.5,2436464 4.0,3546566 6.0,2566566 6.0,2435666 6.0,27.5


In [16]:
df_ranking_finals = df_dict[data_dfs.ranking_list][0].copy()
# Drop the first row which contains only the key word 'Finals'.
df_ranking_finals = df_ranking_finals[1:]
# Set the first row as headder, and drop it form the data body.
df_ranking_finals.columns = df_ranking_finals.iloc[0]
df_ranking_finals = df_ranking_finals[1:]

# Since the placements of the finals are redundant information and will be extracted from the finals table,
# delete it here. In case that there were more rounds this allows to easily join the other rankings to this table.
df_ranking_finals.drop(columns=['LW', 'TG', 'WW', 'SF', 'QU', 'SB', 'CC', 'RB', 'PD', 'JV', 'PZ'], inplace=True, errors='ignore')
# Split the club name from the couples name (and their number).
df_ranking_finals[[c.C.COUPLE.value, c.C.CLUB.value]] = df_ranking_finals['Paar/Club'].str.split(")", expand = True)
df_ranking_finals.drop(columns=['Paar/Club'], inplace=True)
df_ranking_finals[[c.C.COUPLE.value, c.C.NR.value]] = df_ranking_finals[c.C.COUPLE.value].str.split("(", expand = True)
df_ranking_finals[['Round']] = 'Endrunde'
df_ranking_finals = df_ranking_finals[[c.C.PLACEMENT.value, 'Round', c.C.NR.value, c.C.COUPLE.value, c.C.CLUB.value]]
df_ranking_finals

1,Platz,Round,Nr.,Paar,Club
2,1.,Endrunde,6,"Spörl, Sebastian / Rodionova, Kristina",TSC Rot-Gold-Casino Nürnberg
3,2.,Endrunde,16,"Brinkmann, Christian / Ostholt, Alena",Die Residenz Münster
4,3.,Endrunde,7,"Otto, Sascha / Vollmer, Janne",TC Der Frankfurter Kreis
5,4.,Endrunde,2,"Ciomber, Dennis / von Brühl, Julia","Art of Dance, Köln"
6,5.,Endrunde,12,"Bohn, Bruno / Bohn, Monika",TC Schwarz-Weiß Reutlingen
7,6.,Endrunde,14,"Rybka, Harald / Rybka-Schmidt, Sarah",TC Rot-Weiss Casino Mainz


In [17]:
df_ranking_other = df_dict[data_dfs.ranking_list][1].copy()
# Drop all rows which contain only NaN values. These are originally used to separate the tables.
df_ranking_other.dropna(axis=0, how='all', inplace=True)
# Rename the columns.
df_ranking_other.columns = [c.C.PLACEMENT.value, c.C.COUPLE.value, c.C.CLUB.value]

# Split the club name from the couples name (and their number).
df_ranking_other[[c.C.NR.value]] = df_ranking_other[c.C.COUPLE.value].str.extract(r'\((\d*)\)', expand = True)
df_ranking_other[[c.C.COUPLE.value]] = df_ranking_other[c.C.COUPLE.value].str.extract(r'(.*)\(', expand = True)

# Collect the row indices that contain names instead of placements. These are the round names.
round_rows = df_ranking_other[df_ranking_other[c.C.PLACEMENT.value].str.endswith('runde')]
round_list: List[Tuple[int, str]] = [(i, round_rows[c.C.PLACEMENT.value][i]) for i in round_rows.index] + [(len(df_ranking_other), '')]

# Mark all following rows with the corresponding round name in a new column.
df_ranking_other[['Round']] = round_list[0][1]
for i in range(len(round_list) - 1):
    start_index, round_name = round_list[i]
    end_index = round_list[i+1][0]
    df_ranking_other.loc[start_index : end_index + 1, 'Round'] = round_name


# Now delete the rows with the round names. 
# Notice that we ignore the last entry since this is an artificial one to finish the for-loop above.
for i, _ in round_list[:-1]:
    df_ranking_other.drop(i, inplace=True)

# Rearrange the column order.
df_ranking_other = df_ranking_other[[c.C.PLACEMENT.value, 'Round', c.C.NR.value, c.C.COUPLE.value, c.C.CLUB.value]]

df_ranking_other

,Platz,Round,Nr.,Paar,Club
2,7.,1. Zwischenrunde,17,"Turpin, Mark / Turpin, Sina",TC Rot-Weiss Casino Mainz
3,8.,1. Zwischenrunde,5,"Steinhäuser, Arnd / Lotsch, Martina",TanzSportZentrum Braunschweig
4,9.- 10.,1. Zwischenrunde,3,"Nölle, Jochen / Flohr-Nölle, Tatjana",Tanzsportclub Dortmund
5,9.- 10.,1. Zwischenrunde,13,"Müller, René / Pfeffer, Anna Maria",TSC Savoy München
6,11.,1. Zwischenrunde,66,"Weber, Christian / Weber, Kim",TanzZentrum Ludwigshafen
7,12.,1. Zwischenrunde,1,"Wilhelm, Florian / Friederich, Yvonne",TSA d. 1. SC Norderstedt
10,13.,Vorrunde,20,"Knittel, Gordon / Rührmund, Isabelle",btc Grün-Gold der Turngemeinde in Berlin 1848
11,14.,Vorrunde,18,"Walz, Marco / Hopfgarten, Christina",TSG Leverkusen
12,15.,Vorrunde,8,"Schröder, Karsten / Rothe, Christine",Askania - TSC Berlin
13,16.- 17.,Vorrunde,9,"Adrian, Claudius / Fischer-Adrian, Andrea",TSC Rot-Gold-Casino Nürnberg


In [18]:
df_rankings = pd.concat([df_ranking_finals, df_ranking_other])
df_dict[data_dfs.ranking_list] = df_rankings
df_dict[data_dfs.ranking_list].to_csv(Path(competition_dir, 'ranking_list.csv'))
df_dict[data_dfs.ranking_list]

,Platz,Round,Nr.,Paar,Club
2,1.,Endrunde,6,"Spörl, Sebastian / Rodionova, Kristina",TSC Rot-Gold-Casino Nürnberg
3,2.,Endrunde,16,"Brinkmann, Christian / Ostholt, Alena",Die Residenz Münster
4,3.,Endrunde,7,"Otto, Sascha / Vollmer, Janne",TC Der Frankfurter Kreis
5,4.,Endrunde,2,"Ciomber, Dennis / von Brühl, Julia","Art of Dance, Köln"
6,5.,Endrunde,12,"Bohn, Bruno / Bohn, Monika",TC Schwarz-Weiß Reutlingen
7,6.,Endrunde,14,"Rybka, Harald / Rybka-Schmidt, Sarah",TC Rot-Weiss Casino Mainz
2,7.,1. Zwischenrunde,17,"Turpin, Mark / Turpin, Sina",TC Rot-Weiss Casino Mainz
3,8.,1. Zwischenrunde,5,"Steinhäuser, Arnd / Lotsch, Martina",TanzSportZentrum Braunschweig
4,9.- 10.,1. Zwischenrunde,3,"Nölle, Jochen / Flohr-Nölle, Tatjana",Tanzsportclub Dortmund
5,9.- 10.,1. Zwischenrunde,13,"Müller, René / Pfeffer, Anna Maria",TSC Savoy München


## Processing of the finals

In [19]:
df_finals = df_dict[data_dfs.finals][0]
dance_name_rows = df_finals[df_finals[0].isna()]
dance_name_ids = dance_name_rows.index.to_list() + [len(df_finals.index)]

# Add a column to display the dance for which the rankings are for. 
# This will replace the horizonal headlines indicating the dances - which will be deleted afterwards.
df_finals[['Dance']] = ''
df_finals['Dance'].iloc[1] = 'Dance'
for i in range(len(dance_name_ids) - 1):
    start_index = dance_name_ids[i]
    dance_name = c.parse_dance_name(dance_name_rows.loc[start_index, 1])
    end_index = dance_name_ids[i+1] - 1
    df_finals.loc[start_index:end_index, 'Dance'] = dance_name

# Now delete the rows with the dance names. 
# Notice that we ignore the last entry since this is an artificial one to finish the for-loop above.
for i in dance_name_ids[:-1]:
    df_finals.drop(i, inplace=True)

# Replace the couple names by only their number.
df_finals[[c.C.NR.value]] = df_finals.iloc[:, 0].str.extract(r'(\d+)\D*', expand = True)
df_finals[c.C.NR.value].iloc[1] = c.C.NR.value
df_finals = df_finals.iloc[:, 1:]

# Replace the adjudicator names by only their reference letter.
for i, value in enumerate(df_finals.iloc[1, :]):
    if value == "1.": 
        break

    df_finals.iloc[1, i] = value[:1]

# Delete the first line which only denotes where the adjudicators and results columns are - same as the second row does.
df_finals = df_finals.iloc[1:]
# Set the now first row as header.
df_finals.columns = df_finals.iloc[0]
df_finals = df_finals[1:]

df_dict[data_dfs.finals] = df_finals
df_dict[data_dfs.finals].to_csv(Path(competition_dir, 'finals.csv'))
df_dict[data_dfs.finals]

1,A,B,C,D,E,F,G,1.,1.-2.,1.-3.,1.-4.,1.-5.,1.-6.,Platz,Summe,Dance,Nr.
3,5,6,5,4,3,4,5,0,0,1,3,6,-,40,40,LW,2
4,1,2,1,2,1,1,1,5,-,-,-,-,-,10,10,LW,6
5,4,4,4,3,2,2,2,0,3,4,-,-,-,30,30,LW,7
6,6,3,6,5,6,3,4,0,0,2,3,4(15),7(33),55,55,LW,12
7,3,5,3,6,4,6,6,0,0,2,3,4(15),7(33),55,55,LW,14
8,2,1,2,1,5,5,3,2,4,-,-,-,-,20,20,LW,16
10,6,6,6,3,3,3,6,0,0,3,3,3,7,60,100,TG,2
11,1,1,1,2,1,1,1,6,-,-,-,-,-,10,20,TG,6
12,4,5,4,5,2,4,2,0,2,2,5(16),-,-,30,60,TG,7
13,5,3,5,4,6,2,5,0,1,2,3,6,-,50,105,TG,12


## Processing of the qualifications table

In [63]:
df_qualifications = df_dict[data_dfs.qualifications][0]
n: int = len(df_dict[data_dfs.qualifications])
if n > 1:
    for i in range(1, n):
        # Drop the first column since it is already present from the first dataframe.
        df_tmp = df_dict[data_dfs.qualifications][i].iloc[:, 1:]
        # Append the dataframe to the rest.
        df_qualifications = pd.concat([df_qualifications, df_tmp], axis=1)

# Delete the first row since it does not really contain information
df_qualifications = df_qualifications[1:]
# Because of the concatenation the column indices could be messed up. Reset them to a range.
df_qualifications.columns = range(len(df_qualifications.columns))

# Delete all rows where all values are NaN.
df_qualifications.dropna(axis=1, how='all', inplace=True)

# Depending on the size of the tournament, the names of the competitors are perhaps repeated.
# In this case frop this row since it is redundant.
df_qualifications.drop(df_qualifications.loc[df_qualifications.iloc[:, 0] == 'Startnummer'].index, inplace=True)


# Replace the adjudicator names by only their reference letter.
# Replace the couple names by only their number.
couple_nrs = df_qualifications.iloc[0, :].str.extract(r'(\d+)\D*')
df_qualifications = pd.concat([couple_nrs.T, df_qualifications], axis=0, ignore_index=True)
df_qualifications.iloc[0, 0] = c.C.NR.value
df_qualifications.columns = df_qualifications.iloc[0]
df_qualifications.drop([0, 1], inplace=True)
    
df_qualifications

,Nr.,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,66
2,A) Petra Matschullat-Horn B) Christoph Losensk...,1 1 5 1 2 5 0,4 3 5 4 5 5 5,4 4 5 4 5 4 3,2 0 0 1 0 0 2,4 5 5 5 4 5 1,5 5 5 5 5 5 5,5 5 5 5 5 5 5,1 1 1 2 0 4 1,1 0 0 1 3 0 4,...,5 5 4 5 5 5 5,0 4 4 0 3 1 5,5 5 5 5 5 5 5,0 0 0 0 0 0 0,5 5 5 5 5 5 5,5 5 5 5 5 2 5,1 4 0 2 3 0 2,5 0 4 0 0 0 0,0 4 1 3 1 4 0,5 4 1 5 4 5 4
3,Ergebnis der Vorrunde,15,31,29,5,29,35,35,10,9,...,34,17,35,0,35,32,12,9,13,28
4,Qualifiziert für die 1. Zwischenrunde,X,X,X,NaN,X,X,X,NaN,NaN,...,X,X,X,NaN,X,X,NaN,NaN,NaN,X
5,A) Petra Matschullat-Horn B) Christoph Losensk...,0 0 0 0 0 0 0,3 1 4 4 5 5 5,0 3 0 0 1 0 0,NaN,0 0 2 2 0 1 3,5 5 5 5 5 5 5,5 5 5 4 5 5 5,NaN,NaN,...,3 5 4 5 3 5 1,0 0 0 0 1 0 3,5 2 5 0 4 4 3,NaN,5 5 5 5 3 5 5,4 4 0 5 2 0 0,NaN,NaN,NaN,0 0 0 0 1 0 0
6,Ergebnis der 1. Zwischenrunde,0,27,4,NaN,8,35,34,NaN,NaN,...,26,4,23,NaN,33,15,NaN,NaN,NaN,1
7,Qualifiziert für die Endrunde,NaN,X,NaN,NaN,NaN,X,X,NaN,NaN,...,X,NaN,X,NaN,X,NaN,NaN,NaN,NaN,NaN
8,Addition Endrunde,NaN,210,NaN,NaN,NaN,50,170,NaN,NaN,...,245,NaN,275,NaN,100,NaN,NaN,NaN,NaN,NaN
9,Platz von Platz bis,12,4,9 10,19,8,1,3,15,16 17,...,5,9 10,6,20,2,7,14,16 17,13,11
10,Aufstiegspunkte,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
